In [ ]:
# Run this cell to set up packages for lecture.
from lec09_imports import *

# Lecture 9 – Grouping on Multiple Columns, Merging

## DSC 10, Winter 2025

### Agenda

- Grouping on multiple columns.
- Merging.

## Grouping on multiple columns

### DSC 10 student data

In [ ]:
roster = bpd.read_csv('data/roster-anon.csv')
roster

Recall, last class, we extracted the first name of each student in the class.

In [ ]:
def first_name(full_name):
    '''Returns the first name given a full name.'''
    return full_name.split(' ')[0]

In [ ]:
roster = roster.assign(
    first=roster.get('name').apply(first_name)
)
roster

### How many students named Ryan are in each section?

We discovered that Ryan and Andrew were tied for the most popular first names overall.

In [ ]:
name_counts = (
    roster
    .groupby('first')
    .count()
    .sort_values('name', ascending=False)
    .get(['name'])
)
name_counts

To find the number of `'Ryan'`s in each lecture section, we can query for only the rows corresponding to `'Ryan'`s, and then group by `'section'`.

In [ ]:
roster[roster.get('first') == 'Ryan'].groupby('section').count()

But what if we want to know the number of `'Vanessa'`s and `'Nathan'`s per section, too?

In [ ]:
roster[roster.get('first') == 'Vanessa'].groupby('section').count()

In [ ]:
roster[roster.get('first') == 'Nathan'].groupby('section').count()

Is there a way to do this **for all first names and sections** all at once?

### How many students with each first name does each lecture section have?

- Right now, we can count the number of students with each first name, by grouping `roster` by `'first'`.

In [ ]:
# One row per unique first name.
roster.groupby('first').count().get(['name'])

- We can also count the number of students in each lecture section, by grouping `roster` by `'section'`.

In [ ]:
# One row per unique section.
roster.groupby('section').count().get(['name'])

- However, neither of the above DataFrames give us the number of students **with each first name** **in each section**.
    - For instance, neither result tells me the number of `'Ryan'`s in the 9AM section or the number of `'Vanessa'`s in the 10AM section.

- It would be nice if we could group by both `'first'` **and** `'section'` – and we can!

### Grouping on multiple columns

In [ ]:
roster

We can pass a **list** of column names to `.groupby`!

In [ ]:
roster.groupby(['section', 'first']).count()

The above DataFrame is telling us, for instance, that there is 1 student with the first name `'Zora'` in the 11AM section.

It is **not** saying that there is only one `'Zora'` in the course overall. There could be more in the other sections.

### Grouping on multiple columns

- To group on multiple columns, pass a **list** of column names to `.groupby`:

<br>

<center><code>df.groupby(['col_1', 'col_2', ..., 'col_k'])</code></center>


- Group by `'col_1'` first. **Within each group**, group by `'col_2'`, and so on.

- **Important: The resulting DataFrame has one row per unique combination of entries in the specified columns.**
    - On the previous slide, we had exactly one row for every combination of `'section'` and `'first'`.

- Formally, when we group on multiple columns, we are creating _subgroups_ – that is, **groups within groups**.
    - On the previous slide, we first grouped by `'section'`, and within each section, we grouped by `'first'`.

### Notice the index... 🤔

- This is called a "[MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)".
    - The DataFrame is indexed by `'section'` **and** `'first'`.
- We won't worry about the details of MultiIndexes.
- We can use `.reset_index()` to "flatten" our DataFrame back to normal.

In [ ]:
roster.groupby(['section', 'first']).count().reset_index()

### Does order matter?

In [ ]:
roster.groupby(['section', 'first']).count().reset_index()

In [ ]:
roster.groupby(['first', 'section']).count().reset_index()

**Answer**: Kind of. The order of the rows and columns will be different, but the content will be the same.

### Activity

1. Using `counts`, find the lecture section with the most `'Amelia'`s.
2. Using `counts`, find the longest first name in the class that is shared by at least two students in the same section.

<details>
<summary>✅ Click <b>here</b> to see the solutions <b>after</b> you've tried both problems.</summary>
    
<pre>

amelia_counts = counts[counts.get('first') == 'Amelia']
amelia_counts.sort_values('name', ascending=False).get('section').iloc[0]

with_len = counts.assign(length=counts.get('first').apply(len))
with_len[with_len.get('name') >= 2].sort_values('length', ascending=False).get('first').iloc[0]

</pre>
        
</details>

In [ ]:
counts = roster.groupby(['section', 'first']).count().reset_index()
counts

### Example: Sea temperatures 🌊

[This dataset](https://library.ucsd.edu/dc/object/bb4003017c) contains the sea surface temperature in La Jolla, on many days ranging from August 22, 1916 to June 30, 2024.

In [ ]:
sea_temp = bpd.read_csv('data/sea_temp.csv')
sea_temp

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

We want to find the single month (e.g. November 1998) with the highest average `'SURFACE_TEMP'`.

Which of the following would help us achieve this goal?

A. `sea_temp.groupby('SURFACE_TEMP').mean()`

B. `sea_temp.groupby('MONTH').mean()`

C. `sea_temp.groupby(['YEAR', 'MONTH']).mean()`

D. `sea_temp.groupby(['MONTH', 'DAY']).mean()`

E. `sea_temp.groupby(['MONTH', 'SURFACE_TEMP']).mean()`

In [ ]:
...

### Plots of monthly and yearly average surface temperature 📈

In [ ]:
(sea_temp
 .groupby('MONTH') 
 .mean() 
 .plot(kind='line', y='SURFACE_TEMP')
);

In [ ]:
# Why is there a sudden drop at the end? Look at the dates of data collection!
(sea_temp
 .groupby('YEAR') 
 .mean() 
 .plot(kind='line', y='SURFACE_TEMP')
);

### Summary: Grouping on multiple columns

- Pass a list of columns to `.groupby` to group on multiple columns. This creates groups within groups.
- Use `.reset_index()` after grouping on multiple columns to move the MultiIndex back to the columns.

## Merging 🚙

In [ ]:
offer_percentage = bpd.DataFrame().assign(
    clothing_type=['Shirt', 'Pants', 'Dress', 'Shorts', 'Shoes'],
    offer_percentage=[20, 30, 50, 30, 50]
)

clothes = bpd.DataFrame().assign(
    item=['Dress', 'Shirt', 'Shoes', 'Pants', 'Shoes'],
    retail_price=[150, 30, 90, 50, 70]
)

### Example: Clothing Resale 👕

In [ ]:
# The percentage of retail price that I can earn for reselling my clothes.
offer_percentage

In [ ]:
# The items I want to sell and their retail prices.
clothes

- **Question**: If I sell all of the clothes in my collection, how much will I earn?

- **Issue**: The information I need to answer the question is spread across multiple DataFrames.

### If I sell all of the clothes in my collection, how much will I earn?

In [ ]:
clothes_merged = offer_percentage.merge(clothes, left_on='clothing_type', right_on='item')
clothes_merged

### What just happened!? 🤯

In [ ]:
# Click through the presentation that appears.
merging_animation()

### `.merge`

To "merge" two DataFrames:
- Pick a "left" and "right" DataFrame.
- Choose a column from each to "merge on".
```py
left_df.merge(
    right_df, 
    left_on='left_col_name',
    right_on='right_col_name'
)
```
- `left_on` and `right_on` should be column names (they don't have to be the same).
- The resulting DataFrame contains a single row for every match between the two columns.
- Rows in either DataFrame without a match disappear!

### If I sell all of the clothes in my collection, how much will I earn?

In [ ]:
clothes_merged = offer_percentage.merge(clothes, left_on='clothing_type', right_on='item')
clothes_merged

In [ ]:
# If I sell all of the clothes in my collection, how much will I earn?
(clothes_merged.get('offer_percentage') / 100 * clothes_merged.get('retail_price')).sum() 

### Does it matter which DataFrame is the left or right DataFrame? 🤔

In [ ]:
offer_percentage.merge(clothes, left_on='clothing_type', right_on='item')

In [ ]:
clothes.merge(offer_percentage, left_on='item', right_on='clothing_type')

**Answer**: The order of the rows and columns will be different, but the content will be the same.

### Special cases

#### What if the names of the columns we want to merge on are both the same?

Instead of using `left_on='col'` and `right_on='col'`, you can just say `on='col'`.

In [ ]:
offer_percentage

In [ ]:
clothes_relabeled = clothes.assign(clothing_type=clothes.get('item')).drop(columns=['item'])
clothes_relabeled

In this example, the column we want to merge on in both DataFrames is named `'clothing_type'`, so we can just use `on='clothing_type'`.

In [ ]:
offer_percentage.merge(clothes_relabeled, on='clothing_type')

**Notice**: There's only one column containing the type of clothing now.

#### What if we want to merge using an index instead of a column?

Instead of using `left_on` or `right_on`, use `left_index=True` or `right_index=True`.

In [ ]:
offers_by_item = offer_percentage.set_index('clothing_type')
offers_by_item

In [ ]:
clothes

In this example, we want to merge using the index in the left DataFrame (`offers_by_item`) and the `item` column in the right DataFrame (`clothes`).

In [ ]:
offers_by_item.merge(clothes, left_index=True, right_on='item')

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

How many rows are in `weather.merge(schools, on='city')`?

    A. 4    B. 5    C. 6    D. 7    E. 8

In [ ]:
concept_check()

### Followup activity

How many rows are in `weather.merge(schools, on='state')`?

### More practice!

Here are related exam problems to help you practice merging:
- [Problem 5 from the Fall 2021 midterm](https://practice.dsc10.com/fa21-midterm/index.html#problem-5).
- [Problem 7 from the Fall 2022 midterm](https://practice.dsc10.com/fa22-midterm/index.html#problem-7).

## Summary, next time

### Summary

- To group on multiple columns, pass a list to `.groupby`.
    - The result has one row for every unique combination of elements in the specified columns.
- To combine information from multiple DataFrames, use `.merge`.
    - When using `.merge`, Python searches for a match between a specified column in each DataFrame and combines the rows with a match.
    - If there are no matches, the row disappears!

### Next time

- **`if`-statements**, to execute code only when certain conditions are met.
- **`for`-loops**, to repeat code many times.
- Both are foundational programming tools. 🛠